# Approximate Killing Vector Fields on Meshes

Approximate Killing vector fields (AKVFs) produce near-isometric deformations.
This is a simple implementation of AKVFs in `python` based loosely on [As-Killing-As-Possible Vector Fields for Planar Deformation](https://people.csail.mit.edu/jsolomon/assets/kvf_deformation.pdf)
This script generalizes from 2D to 3D, but does not use the two-level optimization or the logarithmic spiral trajectories.

The Killing energy of a domain $\Omega$ (e.g. 2D surface embedded in 3D) deformed along the deformation field $U$ is
$$ E_K (U) = \int_{p \in \Omega} || J_U(p) +J_U(p)^\top ||^2 $$

Given a set of user-prescribed velocities $u_i$ at locations $C=\{ p_i \}$ the as-Killing-as-possible deformation field $U$ is found by minimizing the Killing energy and constraint violation:
$$ U^0 = \argmin \left( E_K(U) + \lambda \sum_{p_i \in C} U(p_i) - u_i \right) $$

However, this creates singularities around the constraints. We regularize the solution using the deformed boundary as a constraint
$$ U_1 = \argmin \left( E_K(U) \text{ s.t. } U=U^0 \text{ on } \delta\Omega \right) $$

In practice, we discretize the quantities and operators on a mesh. For $n$ vertices and $m$ faces:
- The velocity $U \in \R^{n \times 3}$.
- The gradient $G \in \R^{3m \times n}$ discretizes $\nabla f = G f$ (mapping vertex to face values)
- 

In [1]:
import igl
import scipy as sp
import numpy as np
import k3d

In [2]:
## Import mesh
v, f = igl.read_triangle_mesh("data/bunny.obj")
n = len(v)
m = len(f)

In [3]:
## Define some constraints
fig = k3d.plot() 
fig += k3d.mesh(v, f, color=0, wireframe=True)

## EAR
pt_ear = np.array([-.02, .17, -.015])
r_ear = 0.03
idxs_ear = np.where(np.square(v-pt_ear).sum(1) < np.square(r_ear))[0]
dv_ear = np.zeros([len(idxs_ear), 3]) + [.01,0,.01]

fig += k3d.points(pt_ear, color=0x0000ff, point_size=2*r_ear, opacity=.5)
fig += k3d.points(v[idxs_ear], color=0x0000ff, point_size=0.001)

## BASE
idxs_base = np.where(v[:,1] < (v.min(axis=0)[1] + .002))[0]
dv_base = np.zeros([len(idxs_base), 3])
fig += k3d.points(v[idxs_base], color=0xff0000, point_size=0.001)

idxs = np.hstack([idxs_ear, idxs_base])
dv = np.zeros(v.shape)
dv[idxs_ear] = dv_ear
dv[idxs_base] = dv_base

fig.display()

/home/arturs/anaconda3/envs/py38/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/arturs/anaconda3/envs/py38/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [54]:
## Discrete gradient operator
G = igl.grad(v, f)  ## shape (3*|F|, |V|)
## Split into gradients wrt x,y,z 
Gx, Gy, Gz = G[:m], G[m:2*m], G[2*m:]

## P is the discrete operator taking vertices to Killing energy in Forbenius norm
## Each row corresponds to a different term when expanding ||J+J'||^2

S = np.sqrt(2)
P = sp.sparse.bmat([
    [2*Gx, None, None],    ## (2*du_x/dx)^2
    [None, 2*Gy, None],    ## (2*du_y/dy)^2
    [None, None, 2*Gz],    ## (2*du_y/dz)^2
    [S*Gy, S*Gx, None],    ## 2*(du_x/dy + du_y/dx)^2
    [S*Gz, None, S*Gx],    ## 2*(du_y/dz + du_z/dy)^2
    [None, S*Gz, S*Gy],    ## 2*(du_x/dx + du_z/dx)^2
])

## Due to how this block matrix is created, every column represents a component of du.
## That means, that the unknown du is a stacked vector [du_x, du_y, du_z].
## This corresponds to flattening and reshaping with order='F' as opposed to 'C'.

In [59]:
## Minimize via lstsq ##
lmbd = 1e2
k = len(idxs)
I = sp.sparse.csr_matrix((k,n), dtype=np.float64)
for i, idx in enumerate(idxs):
    I[i, idx] = 1
Ik = sp.sparse.bmat([
    [1*I,0*I,0*I],
    [0*I,1*I,0*I],
    [0*I,0*I,1*I]
])

A = sp.sparse.bmat([[P], [lmbd*Ik]])
b = np.hstack([np.zeros([P.shape[0]]), lmbd*dv[idxs].flatten(order='F')])


sol = sp.sparse.linalg.lsqr(A, b)
du = sol[0].reshape(v.shape, order='F')

print(np.abs(du).max()) ## Sanity check

0.009905415218494787


In [60]:
import k3d

fig = k3d.plot() 

# fig += k3d.mesh(cs, f, color=0x00ff00, attribute=np.sqrt(np.square((cs-v)).sum(1)))
# fig += k3d.mesh(u, f, color=0x00ff00, attribute=np.sqrt(np.square((u-cs)).sum(1)))
# for i in range(800,1000):
#     fig += k3d.text(str(i), v[i].tolist(), reference_point='cc', label_box=False, color=0)

fig += k3d.mesh(v, f, color=0, wireframe=True)
fig += k3d.mesh(v+du, f, attribute=np.sqrt(np.square(du).sum(1)), opacity=.8)



fig.display()

Output()

In [61]:
## Find the Killing energy on each face
e_K = np.square(P@sol[0]).reshape(len(f),-1, order='F').sum(-1)

In [62]:
import meshplot as mp

mp.plot(v+du, f, e_K)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014101…